In [ ]:
import json
from googleapiclient import discovery
import pandas as pd
import time

In [ ]:
API_KEY = 'YOUR_API_KEY' # Place your API key here

# Initialize the Perspective API client
service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=API_KEY)

In [ ]:
def analyze_text(text):
    analyze_request = {
        'comment': {'text': text},
        'requestedAttributes': {
            'TOXICITY': {}, 'SEVERE_TOXICITY': {}, 'IDENTITY_ATTACK': {},
            'INSULT': {}, 'PROFANITY': {}, 'THREAT': {}
        },
        'languages': ['en']
    }

    try:
        response = service.comments().analyze(body=analyze_request).execute()
        attribute_scores = response['attributeScores']
        scores_dict = {attribute: scores['summaryScore']['value'] for attribute, scores in attribute_scores.items()}
        return scores_dict
    except Exception as e:
        print(f'Error analyzing text: {e}')
        return None

In [ ]:
data = pd.read_csv('../data/processed/cleaned_transcripted_dataset.csv')

data['full_text'] = data['video_desc'].fillna('') + ' ' + data['video_hashtags'].fillna('') + ' ' + data['stickers_on_video'].fillna('') + ' ' + data['transcription'].fillna('')

In [ ]:
# Analyze text in each row of the 'full_text' column and store the scores in new columns
attributes = ['TOXICITY', 'SEVERE_TOXICITY', 'IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'THREAT']
for i, row in data.iterrows():
    text = row['full_text']
    scores = analyze_text(text)
    if scores:
        for attribute in attributes:
            data.loc[i, attribute] = scores.get(attribute)
    time.sleep(2)  # Add a delay of 2 seconds between requests

In [ ]:
data.head(10)

In [ ]:
# Store data to csv file
data.to_csv('../data/processed/toxicity_analysis_perspective.csv', index=False)